In [1]:
import numpy as np
from PIL import Image

In [11]:
def rgb_to_ycbcr(image):
    """
    Convert an RGB image to YCbCr and return separate Y, Cb, and Cr images.
    Parameters: a PIL Image object in RGB mode.
    Returns: Y, Cb, Cr: numpy arrays representing the Y, Cb, and Cr components.
    """
    img_array = np.array(image, dtype=float)
    
    # Separate the RGB channels
    R = img_array[:, :, 0]
    G = img_array[:, :, 1]
    B = img_array[:, :, 2]
    
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    Cb = 128 - 0.168736 * R - 0.331264 * G + 0.5 * B
    Cr = 128 + 0.5 * R - 0.418688 * G - 0.081312 * B
    
    return Y, Cb, Cr

def save_component_image(component, filename):
    """
    Save a single Y, Cb, or Cr component as an image.
    
    Parameters:
    - component: a numpy array representing the Y, Cb, or Cr component.
    - filename: the name of the file to save the image as.
    """
    component_uint8 = np.uint8(component)
    component_pil = Image.fromarray(component_uint8)
    component_pil.save(filename)

def apply_420_subsampling(Cb, Cr):
    """
    Apply 4:2:0 subsampling to the Cb and Cr components.
    
    Parameters:
    - Cb: numpy array representing the Cb component.
    - Cr: numpy array representing the Cr component.
    
    Returns:
    - Cb_subsampled, Cr_subsampled: numpy arrays representing the subsampled Cb and Cr components.
    """
    # Perform 4:2:0 subsampling
    Cb_subsampled = Cb[::2, ::2]  # Take every second row and column
    Cr_subsampled = Cr[::2, ::2]  # Take every second row and column
    
    return Cb_subsampled, Cr_subsampled

def ycbcr_to_rgb(Y, Cb_subsampled, Cr_subsampled):
    """
    Convert YCbCr components back to RGB.
    
    Parameters:
    - Y: numpy array representing the Y component.
    - Cb_subsampled: numpy array representing the subsampled Cb component.
    - Cr_subsampled: numpy array representing the subsampled Cr component.
    
    Returns:
    - rgb_image: PIL Image object representing the reconstructed RGB image.
    """
    # Upsample Cb and Cr components to match Y size (repeat rows and columns)
    height, width = Y.shape
    Cb_upsampled = np.repeat(np.repeat(Cb_subsampled, 2, axis=0), 2, axis=1)
    Cr_upsampled = np.repeat(np.repeat(Cr_subsampled, 2, axis=0), 2, axis=1)
    
    # Perform inverse YCbCr to RGB conversion
    R = Y + 1.402 * (Cr_upsampled - 128)
    G = Y - 0.344136 * (Cb_upsampled - 128) - 0.714136 * (Cr_upsampled - 128)
    B = Y + 1.772 * (Cb_upsampled - 128)
    
    # Stack R, G, B channels and clip values to [0, 255]
    rgb_image = np.stack((R, G, B), axis=-1)
    rgb_image = np.clip(rgb_image, 0, 255)
    rgb_image = np.uint8(rgb_image)
    
    # Convert numpy array to PIL Image
    rgb_image_pil = Image.fromarray(rgb_image)
    
    return rgb_image_pil
# Example usage:
image_path = 'sample.bmp'
image = Image.open(image_path).convert('RGB')
Y, Cb, Cr = rgb_to_ycbcr(image)

# Save the Y, Cb, and Cr images
# save_component_image(Y, 'Y_component.bmp')
# save_component_image(Cb, 'Cb_component.bmp')
# save_component_image(Cr, 'Cr_component.bmp')

# To visualize the components
Image.fromarray(np.uint8(Y)).show(title="Y Component")
Image.fromarray(np.uint8(Cb)).show(title="Cb Component")
Image.fromarray(np.uint8(Cr)).show(title="Cr Component")


Cb_subsampled, Cr_subsampled = apply_420_subsampling(Cb, Cr)
# Visualize the original and subsampled Cb and Cr components
Image.fromarray(np.uint8(Cb)).show(title="Original Cb Component")
Image.fromarray(np.uint8(Cb_subsampled)).show(title="Subsampled Cb Component")

Image.fromarray(np.uint8(Cr)).show(title="Original Cr Component")
Image.fromarray(np.uint8(Cr_subsampled)).show(title="Subsampled Cr Component")

# Convert YCbCr components back to RGB
reconstructed_rgb_image = ycbcr_to_rgb(Y, Cb_subsampled, Cr_subsampled)

# Display the reconstructed RGB image
reconstructed_rgb_image.show(title="Reconstructed RGB Image")
